In [33]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
es_client = Elasticsearch()
from pprint import pprint

In [34]:
INDEX = "efaqa-70" # index to search e.g. "msmacro-full"
FIELDS = ["pseudo_passage", "pseudo_passage"] # fields to search e.g. ["passage", "query"], ["pseudo_passage", "pseudo_passage"] 

In [35]:
# this is the search result object
class search_results_data:
    def __init__(self, search_session_id, query_input, ranks, qids, pids, query_labels, passages, scores):
        """
        search_session_id - intiger, unique for each search session
        query_input - string, the query
        all of the below should be of the same length and in order
        ranks - list of ranks
        qids - list of qids
        pids - list of pids
        query_labels - list of label queries
        passages - list of passages
        scores - list of scores from search matching
        """
        assert len(ranks)==len(qids)==len(pids)==len(query_labels)==len(passages)==len(scores)
        table = []
        for i, rank in enumerate(ranks):
            table.append({
                "rank": ranks[i],
                "qid": qids[i],
                "pid": pids[i],
                "query_label": query_labels[i],
                "passage": passages[i],
                "score": scores[i],
            })
        self.search_session_id = search_session_id
        self.query_input = query_input
        self.table = table

In [36]:
# we will use elasticsearch-dsl this time; a higher level python es client
# !pip install elasticsearch-dsl

In [37]:
def es_search(query, cutoff = 1000, index=INDEX, fields = FIELDS):
    q = Q({"multi_match": {"query": query, "fields": fields}})

    s = Search(using=es_client, index=index).query(q)

    s.update_from_dict({"size": cutoff})

    response = s.execute()
    response_dict = response.to_dict()
    hits = response_dict['hits']['hits']
    result_count = len(hits)
    return result_count, hits

In [45]:
result_count, hits = es_search("心情", cutoff = 100, index=INDEX, fields = FIELDS)
result_count

100

In [46]:
pprint(hits[0])

{'_id': 'BoDx1HsBVPLpt6nxIWXm',
 '_index': 'efaqa-70',
 '_score': 1.6834424,
 '_source': {'pseudo_passage': '这种情况和心理咨询师交流一下 '
                               '找到问题的根源非常理解你的心情，可能你的个性比较敏感，长期养成了负性思维习惯，凡事爱往坏处想，容易激发抑郁低落，不快乐的心情。建议应加强个人调节，凡事多看好的一面，提高对目前拥有的生活的满意度，缓解内心的不良情绪，增强内心的幸福感。现在这种情况很需要心理咨询',
             'pseudo_query': '什么情况下需要看心理医生，感觉自己一直过得很压抑，不快乐，也能找到不幸福的原因，但找不到解决问题的方法最想得到具体的建议，关于道理或心态的东西自己也了解了很多',
             'short_query': '什么情况下需要看心理医生，感觉自己一直过得很压抑，不快乐，也能找到不幸福的原因，但找不到解决问题的方法'},
 '_type': '_doc'}


In [20]:
def direct_es_search_result(search_session_id, query_input, hits):
    '''
    this parses the search result and returns the search results object
    '''
    ranks, qids, pids, query_labels, passages, scores = [], [], [], [], [], []

    for index, hit in enumerate(hits):
        ranks.append(index)
        qids.append(hit['_source']['qid'])
        pids.append(hit['_source']['pid'])
        query_labels.append(hit['_source']['query'])
        passages.append(hit['_source']['passage'])
        scores.append(hit['_score'])
    
    result = search_results_data(
        search_session_id = search_session_id,
        query_input = query_input,
        ranks = ranks, 
        qids = qids, 
        pids = pids, 
        query_labels = query_labels, 
        passages = passages, 
        scores = scores
    )

    return result


In [21]:
temp = direct_es_search_result(25234, "aksjdf", hits)

In [22]:
pprint(temp.search_session_id)
pprint(temp.query_input)
pprint(temp.table)

25234
'aksjdf'
[{'passage': 'What is the Python Software Foundation?¶. The Python Software '
             'Foundation is an independent non-profit organization that holds '
             'the copyright on Python versions 2.1 and newer. The PSF’s '
             'mission is to advance open source technology related to the '
             'Python programming language and to publicize the use of Python.',
  'pid': 2713990,
  'qid': 267012,
  'query_label': 'how long should i wait to handle my ball python after '
                 'feeding',
  'rank': 1,
  'score': 18.307333},
 {'passage': 'Python Glob. The module glob (in Python) is what you want to be '
             'using if you are trying to use wildcards with Python. '
             'glob.glob(). In case you need to learn or review about wildcards '
             '(in Mac OS X): koldfyre’s tutorial.And here is the official site '
             'with slightly complicated explanations.he module glob (in '
             'Python) is what you want

# some experiment on efaqa-70

In [26]:
result_count, hits = es_search("你", cutoff = 1000, index="efaqa-70", fields = ["pseudo_passage", "pseudo_query"])
result_count

1000

In [27]:
pprint(hits[0])

{'_id': 'MoDx1HsBVPLpt6nxIGK3',
 '_index': 'efaqa-70',
 '_score': 3.4125881,
 '_source': {'pseudo_passage': '您的观点似乎有些 '
                               "''硬''的感觉人贵有自知的能力，找到一个爱自己的，同时自己也爱他的，两全其美只有学会爱自己，才会真正爱别人！一个真正爱自己的人，别人才会爱她，她也才会真实的爱别人你需要心理成长，爱让你投入到忘我，这本身也是不成熟的表现。正所谓君子有所为，有所不为。如果你想成长自己，更成熟一些，可以和我咨询婚姻里如何权衡好爱人与被爱的关系并不容易。把握好亲密关系不是一味的付出或者索取得到的。面对家庭婚姻和孩子，如何成为积极面对生活的人，一切问题就都不会是问题。专业婚恋咨询可以关注我，希望帮能解除困扰。",
             'pseudo_query': '爱你的人无论怎样都会帮你，为你付出，而你爱的人往往是背叛你的人。当你爱一个人很深时，你会发现已经失去自我，迷失方向，当那个人深深伤你，丢下你，你才会发现你的爱是多么的卑微，原来在那一方的爱根本不存在，于是你陷入自责，自伤，你寻求帮助，寻求安慰，可是你却发现只有自己可以相信有时候还是自我一些好吧现在的生活让我出现了恐慌，也对婚姻生活出现了不信任，不知道该让自己如何走。如果没有孩子我会选择离婚，恢复到原来的自己，好好让自己生活',
             'short_query': '爱你的人无论怎样都会帮你，为你付出，而你爱的人往往是背叛你的人。'},
 '_type': '_doc'}


# Try to connect remote es

In [13]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
from pprint import pprint


es_client = Elasticsearch(
    ['47.240.54.55:9200'],
    scheme="https",
    port=29737,
)

In [14]:
INDEX = "ctb-nlp-v1" # index to search e.g. "msmacro-full"
FIELDS = ["passage", "query", "alt_query"] # fields to search in elasticsearch retrieval e.g. ["passage", "query"]
ES_CUTOFF = 100

In [15]:
from es_helper import *

In [16]:
query_input = "随便进儿子房间怎么了吗？"
result_count, es_hits = es_search(query_input, cutoff = ES_CUTOFF, index=INDEX, fields = FIELDS)
es_hits

[{'_index': 'ctb-nlp-v1',
  '_type': '_doc',
  '_id': 'DEk2ZHwBKMvT5r_Rvs-q',
  '_score': 20.451498,
  '_source': {'qid': 'q407',
   'query': '随意进入孩子房间有问题吗？',
   'alt_qid': None,
   'alt_query': None,
   'pid': 'p259',
   'passage': '受访的17名男生中，4名男生报告在小学之前就己经与父母分开睡，并有了自己独立的房间；有12名男生延迟到小学时与父母分房；只有1名男生报告至今仍然经常与母亲同床，理由是因父亲打呼而影响母亲的睡眠；有4名男生报告自己在房间是偶尔需要与父/母亲共用的，有3名的男高中生报告说“只有床是我自己的”，“他总是需要与我共用一个房间”，“他随时进我的房间，也不敲门”；有10名男生会报告说“父母会经常参与到房间内物品的摆放，房间内的打扫等事务”。',
   'citation': '葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2050'}},
 {'_index': 'ctb-nlp-v1',
  '_type': '_doc',
  '_id': '9kk2ZHwBKMvT5r_Rvs6q',
  '_score': 12.868242,
  '_source': {'qid': 'q385',
   'query': '明确的亲子界限是怎么样的？',
   'alt_qid': None,
   'alt_query': None,
   'pid': 'p237',
   'passage': '她大约在3年级的时候便开始一个人睡（书房），初中一年级时有了完全属于自己的房间，父母进到房间之前会先敲门，但一般会在外边喊，叫她出来，房间内卫生需要她自己打扫，爸爸会提供拖地的帮助。“从小到大没有感觉受到侵犯”，当向母亲提出需求（对穿衣品味的审美支持）时，母亲也可以及吋的为其提供帮助。“姥姥姥爷也会尊重我的选择”，在力所能及的情况下，尽量使我满足。',
   'citation': '葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2028'}},
 {'_index': 'ctb-nlp-v1